In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from time import time
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

from mmoe import MMOE
from evaluation import evaluate_deepctr

# GPU相关设置
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Please check the latest version manually on https://pypi.org/project/deepctr/#history


In [2]:
df = pd.read_csv("data/lgb.csv")
df

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,authorid_in_userid_count_prop,userid_bgm_song_id_count,userid_in_bgm_song_id_count_prop,bgm_song_id_in_userid_count_prop,userid_bgm_singer_id_count,userid_in_bgm_singer_id_count_prop,bgm_singer_id_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,...,0.007664,1,0.002487,0.003832,1,0.002487,0.003832,12642.308,10255.0550,5
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,...,0.003832,117,0.000030,0.448200,117,0.000030,0.448200,12642.308,16000.0000,1
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,...,0.003832,117,0.000030,0.448200,117,0.000030,0.448200,12642.308,28736.4940,24
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,...,0.011500,1,0.003311,0.003832,1,0.003311,0.003832,12642.308,6875.7370,17
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,...,0.007664,1,0.000875,0.003832,1,0.000875,0.003832,12642.308,17253.5780,19
5,8,30287,1,1,0.0,0.0,0.0,0.0,1496.0,0.0,...,0.011500,1,0.001279,0.003832,1,0.000403,0.003832,12642.308,20497.5840,38
6,8,1151,1,1,0.0,0.0,0.0,250.0,976.0,0.0,...,0.003832,1,0.006622,0.003832,1,0.006622,0.003832,12642.308,10398.5160,142
7,8,69745,1,1,0.0,0.0,0.0,250.0,817.0,0.0,...,0.003832,1,0.002500,0.003832,1,0.002500,0.003832,12642.308,12257.1390,64
8,8,22451,1,1,0.0,0.0,0.0,0.0,2014.0,0.0,...,0.003832,1,0.001920,0.003832,1,0.000607,0.003832,12642.308,13573.3990,10
9,8,97911,1,1,0.0,0.0,0.0,250.0,861.0,0.0,...,0.007664,1,0.000889,0.003832,1,0.000889,0.003832,12642.308,12580.0610,40


In [3]:
feed_info = pd.read_csv("data/wechat_algo_data1/feed_info.csv")
feed_info

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,NaN,NaN,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...
2,12921,2996,15,114413 107973 117252 27736 41035 32715 125374 ...,NaN,NaN,10288.0,11354.0,NaN,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646,27077 10050 27464 20072 26018 6243 21492 14750...,NaN,NaN
3,34538,12577,18,60473 9864 8109 135558 108803 89307 37630 6481...,39654 95201 22675 49521 135558 108803 89307 37...,7446 47082 37630 99660 9844 113002 9844 113002...,24435.0,13645.0,5454;1197;3727,NaN,328;13;159;6,267 0.21298289;191 0.21298289;8 0.34298885;306...,21489 24672 2223 1882 4956 1545 10424 19791 12...,9015 21209 5026 11192 4956 1545 10424 19791 12...,20825 13395 14765 24513 7305 12837 10337 8885 ...
4,75414,7185,16,105860 4691 134820 55911 80449 79213 23233 139...,73055 66447 88970,104002 4438 104002 4438,23428.0,10730.0,17083;4123,8058;219;21639;15621;25209,81;269;159;6,269 0.86163938;81 0.86163938,25782 19583 7768 3907 14068 12681 5043 17859 1...,30483 23721 1518 22741 25118,23112 969 23112 969
5,34539,2595,43,123046 116187 8092 22091 77895 11717 93522 100...,61498 70163 1428 123046 116187 44971 3063 3214...,147566 45843 10952 25290 79030 11717 34198 993...,16541.0,17296.0,529;2666;26954,529;9298,45;239;159;169,239 0.68520623;45 0.68520623;8 0.0;306 0.0;207...,27442 1543 31605 1868 20857 22179 17181 7261 8...,13842 6439 32379 25274 8847 1251 27442 1543 31...,15371 6498 31376 30024 24307 21492 10083 20072...
6,101310,8115,15,106872 36623 26564 8109 110838 148383 26564 33...,9837 110838 148383 26564 33895 110690 133005,117252 44854 26564 110142 93536 91254 8109 983...,19367.0,10532.0,NaN,18351;13433;25057;11002;14687;10664;19485;12865,9;6,237 0.90296459;246 0.90296459,20831 11704 11598 11354 5967 1882 1217 23805 1...,2200 1217 23805 10083 14040 5967 12194 13395 1...,26018 10155 5967 24513 20833 5311 32214 1882 2...
7,13769,11696,25,144240 54435 37630 117252 27736 40670 134820 1...,63733 106137 31785 73044 25794 65020 90843 107...,123075 11342 79301 23908 110100 112151 110677 ...,NaN,NaN,NaN,8800;11061;15752,9;6,17 0.74826479;37 0.74826479,12783 22473 5277 10040 12837 26018 6243 8311 3...,20857 15640 7261 23039 30060 32499 5009 8481 7...,30191 2606 17583 5391 20861 15640 12836 4895 1...
8,6311,9533,60,102036 39661 66989 110142 118715 93522 24335 8...,122344 140404 55785 26564 93563 102383 27736 7...,90843 31279 117347 116237 8109 287 49575 56976...,NaN,NaN,NaN,17892;18428,9;6,235 0.36773422;204 0.36773422,2682 32028 12649 7461 4913 29499 24513 24368 2...,12676 3901 31177 12653 5967 20857 3653 11877 6...,26018 2223 30031 2220 26054 18659 15360 1882 6...
9,6801,13501,14,45012 25146 45443 122452 875 80983 8109 25146 ...,20248 33368 37767 37767 82007 78435 69591 9354...,90290 77895 25146 106949 129539 106793 304 277...,NaN,NaN,15506;17582;10844,25209;219;4826;25500,81;269;159;6,269 0.96752864;81 0.96752864,10226 13505 15360 10326 17526 26208 253 12762 ...,27393 7536 7349 7349 17323 11893 20838 18755 3...,20085 17181 13505 15360 23786 14009 12831 2706...


In [2]:
feed_embedding = pd.read_csv("data/wechat_algo_data1/feed_embeddings.csv")

In [3]:
feed_embedding.sort_values("feedid", inplace=True)

In [4]:
feed_embedding

,feedid,feed_embedding
99412,0,0.08408414 0.00483176 0.06853063 -0.03777170 -...
13109,1,0.09030288 0.04753714 0.11482160 -0.06640348 0...
97064,3,0.03740683 0.00502505 0.16688652 -0.02305033 0...
67554,4,-0.00898417 -0.01350270 0.11481205 -0.00205122...
44898,6,0.01900294 0.00060241 0.15238224 -0.02654222 -...
53396,7,0.02324197 0.02743889 0.10052337 -0.07085486 -...
7434,8,-0.03293774 0.05535886 0.01709914 0.02070779 0...
48766,10,0.05214350 0.00868368 0.02759746 0.05134032 -0...
92343,11,0.00591145 -0.04763667 -0.01125488 -0.01287244...
36962,12,0.07019624 0.02880802 0.09632652 -0.05703027 0...


In [12]:
rec = []
for i in (set(range(feed_embedding['feedid'].max()+1)) - set(feed_embedding['feedid'])):
    rec.append({"feedid":i, "feed_embedding": feed_embedding.iloc[0, 1]})
rec_df = pd.DataFrame(rec)
df = pd.concat([feed_embedding, rec_df], sort=True)
df

,feed_embedding,feedid
99412,0.08408414 0.00483176 0.06853063 -0.03777170 -...,0
13109,0.09030288 0.04753714 0.11482160 -0.06640348 0...,1
97064,0.03740683 0.00502505 0.16688652 -0.02305033 0...,3
67554,-0.00898417 -0.01350270 0.11481205 -0.00205122...,4
44898,0.01900294 0.00060241 0.15238224 -0.02654222 -...,6
53396,0.02324197 0.02743889 0.10052337 -0.07085486 -...,7
7434,-0.03293774 0.05535886 0.01709914 0.02070779 0...,8
48766,0.05214350 0.00868368 0.02759746 0.05134032 -0...,10
92343,0.00591145 -0.04763667 -0.01125488 -0.01287244...,11
36962,0.07019624 0.02880802 0.09632652 -0.05703027 0...,12


In [13]:
df = df.reset_index(drop=True)
df.sort_values("feedid", inplace=True)
# df['feed_embedding'] = df['feed_embedding'].apply(lambda x: list(map(float, x.strip().split())))
# df['feed_embedding'] = df['feed_embedding'].apply(lambda x: x.replace(" ", ","))
df.to_csv("feed_embedding.csv", index=0)

In [138]:
emb = pd.read_csv("feed_embedding.csv")

In [19]:
feed_embeddings = pd.read_csv("feed_embedding.csv")
feed_embeddings['feed_embedding'] = feed_embeddings['feed_embedding'].apply(
    lambda x: list(map(float, x.strip().split())))
feed_embedding = feed_embeddings['feed_embedding'].values

In [24]:
feed_embedding[0]

[0.08408414,
 0.00483176,
 0.06853063,
 -0.0377717,
 -0.00126421,
 -0.02952033,
 -0.02248864,
 -0.01438222,
 0.0260775,
 -0.10562083,
 0.01603754,
 0.04184927,
 -0.06226169,
 0.03013075,
 0.00145834,
 -0.00356996,
 -0.03820653,
 0.02838919,
 -0.05330689,
 -0.00881395,
 -0.01846378,
 0.04529035,
 -0.05512874,
 -0.03332639,
 -0.02674967,
 0.02755575,
 0.01836011,
 -0.03202691,
 -0.01448034,
 0.01855098,
 0.05449134,
 -0.04612953,
 -0.02035482,
 0.00510818,
 0.02020181,
 -0.05667673,
 -0.03672433,
 -0.0222059,
 -0.00684524,
 -0.07546107,
 -0.00788517,
 0.01400762,
 -0.09180802,
 -0.03593544,
 0.00658322,
 -0.03735572,
 -0.03680798,
 -0.06186584,
 -0.04189884,
 -0.04162169,
 0.10271495,
 0.02981273,
 -0.03328977,
 -0.03891595,
 0.00191436,
 -0.00312964,
 0.02889538,
 0.00084027,
 -0.08082388,
 0.0335694,
 -0.01727188,
 -0.0309547,
 -0.00173673,
 -0.00673804,
 -0.0922366,
 0.01073073,
 0.01192468,
 0.01622597,
 -0.00953173,
 -0.0981409,
 0.11012837,
 0.04287522,
 0.04702039,
 -0.00285473,
 

In [27]:
np.array(feed_embedding.tolist())
    

array([[ 0.08408414,  0.00483176,  0.06853063, ..., -0.08747845,
        -0.09291192,  0.06132171],
       [ 0.09030288,  0.04753714,  0.1148216 , ..., -0.01533513,
        -0.04385881,  0.03210009],
       [ 0.08408414,  0.00483176,  0.06853063, ..., -0.08747845,
        -0.09291192,  0.06132171],
       ...,
       [-0.04885333,  0.01471811,  0.08409678, ...,  0.0253159 ,
        -0.03632009, -0.01606419],
       [ 0.0538084 , -0.07112316,  0.02450147, ..., -0.0457488 ,
        -0.0807632 , -0.03537565],
       [-0.04117391, -0.05257145,  0.09154589, ..., -0.07498139,
        -0.04097828,  0.01641754]])

In [11]:
pretrained_weights = np.random.randn(feed_embedding['feedid'].max() + 1, 512)
pretrained_weights

array([[-0.03671631,  0.22837689,  2.05220026, ..., -0.42029438,
         1.12709307, -0.37850088],
       [ 1.13957025, -0.82078929, -0.54273098, ...,  1.02497951,
        -1.58131283,  0.24799871],
       [-0.19878876, -0.44206375,  0.64826518, ...,  0.60544058,
        -0.96878879, -2.25916204],
       ...,
       [ 1.46758232, -0.79510687,  1.08135231, ..., -1.31940345,
        -0.84600118,  1.47100479],
       [-0.89817373, -1.32808533,  0.7039034 , ..., -0.53818513,
         1.94467456,  0.57811016],
       [ 1.54805856,  0.1381536 , -0.62303059, ...,  0.64176619,
         0.310011  , -0.87762763]])

In [64]:
pd.DataFrame(pretrained_weights)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.491157,-0.073891,0.014678,0.057683,0.643008,0.680161,-0.321174,-0.585853,0.891021,1.564030,...,-0.508416,-0.394305,1.539185,-2.719708,-0.047523,0.273527,1.344624,0.778913,-0.501397,-0.334324
1,-0.439106,0.533734,-0.093868,0.929328,-1.846977,-2.132984,-0.018320,1.295529,0.652674,0.581472,...,1.695286,0.385684,0.107024,-2.096827,-0.309984,-1.099403,1.106001,-0.818055,0.315508,0.128271
2,3.362630,-0.330957,0.178355,-0.595832,0.856238,0.469834,0.215767,-0.150084,-0.914147,-0.561413,...,-1.594034,0.288557,0.166502,-0.284557,0.045339,0.342329,-0.405535,-0.601648,0.462818,-0.491074
3,0.283371,-0.971979,1.082312,-0.591896,-1.169609,1.731758,-0.489171,0.129006,0.553113,-1.603349,...,-1.127667,-0.006429,1.497936,0.679652,-0.656500,-1.449112,-0.455030,1.593951,0.210612,0.154240
4,-0.660497,2.271949,0.332997,1.158634,1.750591,-0.771851,-1.141500,0.622517,-2.068933,0.978399,...,-0.663699,0.299580,-0.653009,-1.059201,0.112341,0.675237,0.658790,0.349197,0.553419,0.418663
5,0.287354,1.535688,0.114409,-0.936184,-0.828614,-0.370845,-0.845198,1.166867,1.508158,0.292746,...,-0.873362,1.251137,0.458782,-0.405314,0.575692,0.491129,0.785040,-0.914894,0.030434,-0.416527
6,0.495539,-0.673328,-0.632120,0.593531,1.084691,-1.873140,0.614035,-2.106440,-0.496880,-0.140286,...,-1.780555,0.880229,-1.306603,-0.437538,-0.881461,1.123216,-1.381131,-1.422942,0.383105,1.414064
7,2.032940,-0.997143,-0.887274,0.841296,0.938368,-0.444710,1.082843,0.119603,-0.540685,-0.169044,...,-0.248439,2.231881,-0.840102,-0.853881,1.477331,0.630097,0.294662,0.658444,1.264830,-0.738031
8,0.706406,0.384715,-0.174115,1.506833,1.526846,1.286355,0.929795,-0.205942,-0.629681,1.070133,...,1.782378,-0.427961,0.810749,0.591713,0.152625,-0.539777,-1.644350,-0.008275,0.332902,0.352908
9,0.937251,0.902312,-1.258305,0.316493,-0.466347,-0.309652,0.572947,-1.243228,-1.016877,-0.974126,...,1.061780,0.260221,-1.225674,0.683999,0.456003,0.387588,0.839404,1.730780,0.565154,-0.472873


In [48]:
feed_embedding['feed_embedding'] = feed_embedding['feed_embedding'].apply(lambda x: np.array(list(map(float, x.strip().split()))))

In [53]:
feed_embedding['feed_embedding'].values[0].shape

(512,)

In [11]:
pretrained_weights = feed_embedding['feed_embedding']

In [7]:
np.random.randn(60, 4)

array([[ 6.26869636e-02, -8.17865677e-01,  3.31697103e-01,
        -2.18706028e-01],
       [-1.54944504e+00, -6.13595911e-01,  9.31682995e-01,
        -1.62244400e-01],
       [ 1.40754625e+00,  6.01995626e-01, -5.02969117e-01,
        -2.42854542e+00],
       [-1.55836569e-01, -7.09333007e-01,  2.32699053e-01,
        -1.23055508e+00],
       [-2.26522301e-01,  1.45620476e+00,  1.45708356e+00,
        -5.62859937e-01],
       [ 9.79746339e-01,  6.71364534e-01, -5.60120295e-01,
        -5.38524641e-01],
       [ 3.76173478e-01, -1.27268166e+00,  2.48503539e-01,
        -1.14463969e+00],
       [ 1.89486553e+00,  5.81246013e-01, -4.57421429e-01,
        -1.59671889e+00],
       [ 1.18052959e+00,  6.09923851e-01, -1.40968179e+00,
         3.25600904e-01],
       [-1.59456083e-01,  1.42918665e+00, -3.72569949e-01,
        -5.15738313e-01],
       [-1.18674928e+00,  1.13326238e+00,  9.65798822e-01,
         5.28012809e-01],
       [-1.75257353e+00, -2.30237726e-01,  2.00804200e-02,
      

In [55]:
pretrained_weights_initializer = tf.initializers.identity(pretrained_weights)

In [56]:
SparseFeat('feedid', 106443, embedding_dim=256, embeddings_initializer=pretrained_weights_initializer)

SparseFeat(name='feedid', vocabulary_size=106443, embedding_dim=256, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.ops.init_ops.Identity object at 0x7fd3d22707f0>, embedding_name='feedid', group_name='default_group', trainable=True)